In [ ]:
# one off operation
!pip install  duckdb --upgrade

In [2]:
import duckdb
import time
def duckdb_attach_lakehouse(workspace,LH):
    con = duckdb.connect(f'temp_{time.time_ns()}.duckdb')
    con.sql('SET enable_object_cache=true')
    ########## get token using notebooks utils inside Fabric
    ########## or a service principals outside Fabric https://fivetran.com/docs/destinations/onelake/setup-guide
    try:
      token = notebookutils.credentials.getToken('storage')
      con.sql(f""" 
                                    CREATE or replace SECRET onelake (
                                    TYPE AZURE,
                                    PROVIDER ACCESS_TOKEN,
                                    ACCESS_TOKEN '{token}') 
                  """)
    except:
        import configparser
        config = configparser.ConfigParser()
        config.read("C:/KV/variable.ini")
        con.sql(f""" 
                                    CREATE or replace SECRET onelake_spn (
                                    TYPE AZURE,
                                    PROVIDER SERVICE_PRINCIPAL,
                                    TENANT_ID     '{config.get("myvars", "tenantId")}',
                                    CLIENT_ID     '{config.get("myvars", "appId")}',
                                    CLIENT_SECRET '{config.get("myvars", "secret") }')
                     """)
    ########## check if the lakehouse has a schema 
    sql_schema     = set()
    sql_statements = set()
    try:
        con.sql(f""" SELECT  * FROM glob ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/dbo/*") """).df()['file'].tolist()
        list_tables = con.sql(f""" SELECT  distinct(split_part(file, '_delta_log', 1)) as tables FROM glob ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/*/*/_delta_log/*.json") """).df()['tables'].tolist()
        for table_path in list_tables:
            parts = table_path.strip("/").split("/")
            schema = parts[-2]
            table = parts[-1]
            sql_schema.add(f"CREATE SCHEMA IF NOT EXISTS {schema};")
            sql_statements.add(f"""CREATE OR REPLACE view {schema}.{table} AS SELECT * 
                                FROM delta_scan('abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/{schema}/{table}');""")
        con.sql(" ".join(sql_schema))
    except:
        list_tables = con.sql(f""" SELECT  distinct(split_part(file, '_delta_log', 1)) as tables FROM glob ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/*/_delta_log/*.json") """).df()['tables'].tolist()
        for table_path in list_tables:
            parts = table_path.strip("/").split("/")
            table = parts[-1]
            sql_statements.add(f"""CREATE OR REPLACE view {table} AS SELECT * 
                                FROM delta_scan('abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/{table}');""")
    con.sql(" ".join(sql_statements))
    con.sql("SHOW ALL TABLES").show(max_width=150)
    con.sql('CHECKPOINT')
    return con

In [3]:
%%time
con = duckdb_attach_lakehouse("aemodev","storage")

┌──────────────────────┬─────────┬─────────────┬──────────────────────┬──────────────────────────────────────────────────────────────────┬───────────┐
│       database       │ schema  │    name     │     column_names     │                           column_types                           │ temporary │
│       varchar        │ varchar │   varchar   │      varchar[]       │                            varchar[]                             │  boolean  │
├──────────────────────┼─────────┼─────────────┼──────────────────────┼──────────────────────────────────────────────────────────────────┼───────────┤
│ temp_1730015519297…  │ main    │ calendar    │ [date, year, month]  │ [TIMESTAMP WITH TIME ZONE, BIGINT, BIGINT]                       │ false     │
│ temp_1730015519297…  │ main    │ duid        │ [DUID, Region, Fue…  │ [VARCHAR, VARCHAR, VARCHAR, VARCHAR]                             │ false     │
│ temp_1730015519297…  │ main    │ mstdatetime │ [SETTLEMENTDATE, t…  │ [TIMESTAMP WITH TIME Z

In [7]:
con.sql(""" 
        -- the summary table will be saved in the local SSD,
        create or replace table summary
        as select date, duid,sum(initialmw)/12 as mw from scada
        where initialmw <> 0
        group by all order by date,duid
        """)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [9]:
df =con.sql(" from summary").df()
df


,DATE,DUID,mw
0,2023-09-28,ALBANY_WF1,1.345002
1,2023-09-28,ALCOA_WGP,-4.359097
2,2023-09-28,ALINTA_PNJ_U1,248.009970
3,2023-09-28,ALINTA_PNJ_U2,260.228640
4,2023-09-28,ALINTA_WGP_GT,0.649861
...,...,...,...
71253,2024-10-27,YARWUN_1,522.722489
71254,2024-10-27,YENDWF1,383.086683
71255,2024-10-27,YWPS2,1469.043453
71256,2024-10-27,YWPS3,1591.117928
